###### ### The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2024 Semester 1

## Assignment 1: Wine quality classification with K-NN


**Student ID(s):**     1297447


This iPython notebook is a template which you will use for your Assignment 1 submission.

**NOTE: YOU SHOULD ADD YOUR RESULTS, DIAGRAMS AND IMAGES FROM YOUR OBSERVATIONS IN THIS FILE TO YOUR REPORT (the PDF file).**

**Adding proper comments to your code is MANDATORY. **

## 0. Preparation

In [4]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.spatial.distance import cdist
from tqdm import tqdm

In [2]:
# Read training dataset from csv file
train_dataset = pd.read_csv('winequality-train.csv')
train_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1350 entries, 0 to 1349
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   fixedAcidity        1350 non-null   float64
 1   volatileAcidity     1350 non-null   float64
 2   citricAcid          1350 non-null   float64
 3   residualSugar       1350 non-null   float64
 4   chlorides           1350 non-null   float64
 5   freeSulfurDioxide   1350 non-null   float64
 6   totalSulfurDioxide  1350 non-null   float64
 7   density             1350 non-null   float64
 8   pH                  1350 non-null   float64
 9   sulphates           1350 non-null   float64
 10  alcohol             1350 non-null   float64
 11  quality             1350 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 126.7 KB


In [3]:
# Read testing dataset from csv file
test_dataset = pd.read_csv('winequality-test.csv')
test_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1350 entries, 0 to 1349
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   fixedAcidity        1350 non-null   float64
 1   volatileAcidity     1350 non-null   float64
 2   citricAcid          1350 non-null   float64
 3   residualSugar       1350 non-null   float64
 4   chlorides           1350 non-null   float64
 5   freeSulfurDioxide   1350 non-null   float64
 6   totalSulfurDioxide  1350 non-null   float64
 7   density             1350 non-null   float64
 8   pH                  1350 non-null   float64
 9   sulphates           1350 non-null   float64
 10  alcohol             1350 non-null   float64
 11  quality             1350 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 126.7 KB


## 1. K-NN classifier

In [28]:
# Seperate features and labels in training and testing dataset
train_features = train_dataset.drop('quality', axis = 1)
train_labels = train_dataset['quality']

test_features = test_dataset.drop('quality', axis = 1)
test_labels = test_dataset['quality']


# Calculate euclidean distance with other features
def calc_euclidean_distance(instance, train_features):
    return np.sqrt(((instance - train_features) ** 2).sum(axis = 1))


# Get neighbors labels from given instance
def get_neighbors_labels(distance, train_labels, k):
    
    # Get k nearest neighbors
    neighbors = distance.sort_values()[:k]
    neighbors_labels = train_labels.loc[neighbors.index].value_counts()

    return neighbors_labels


# Predict one instance's label given k and features
def predict_instance(instance, train_features, train_labels, k):
    
    # Calculate euclidean distance
    distance = calc_euclidean_distance(instance, train_features)
    
    # Get neighbors labels
    neighbors_labels = get_neighbors_labels(distance, train_labels, k)
    
    # Use majority vote to pick the label with most votes
    # If tie, use 1-NN
    if neighbors_labels.get(0) == neighbors_labels.get(1):
        neighbors_labels = get_neighbors_labels(distance, train_labels, k = 1)
        label = neighbors_labels.index[0]
    else:
        label = neighbors_labels.sort_values(ascending = False).index[0]
    
    return label


# Predict the quality of wine using K-NN
def predict(test_features, train_features, train_labels, k):

    predictions = {}

    # Predict each instance in test dataset
    for i, data in test_features.iterrows():
        label = predict_instance(data, train_features, train_labels, k)
        predictions[i] = label

    return pd.Series(predictions)

## 2. 1-NN classification

#### NOTE: you may develope codes or functions to help respond to the question here, but your formal answer must be submitted separately as a PDF.

In [41]:
prediction = predict(test_features, train_features, train_labels, k = 1)
(prediction == test_labels).mean()

0.7644444444444445

## 3. Normalization

#### NOTE: you may develope codes or functions to help respond to the question here, but your formal answer must be submitted separately as a PDF.

## 4. Model extensions

#### NOTE: you may develope codes or functions to help respond to the question here, but your formal answer must be submitted separately as a PDF.

### 4.1
Compare the performance of your best 1-NN model from Question 3 to a Gaussian naive Bayes model on this dataset (you may use library functions to implement the Gaussian naive Bayes model). In your write-up, state the accuracy of the naive Bayes model and identify instances where the two models disagree. Why do the two models classify these instances differently?

### 4.2
Implement two additional distance measures for your K-NN model: cosine similarity and Mahalanobis distance (you may use library functions for these distance measures). Do 1-NN classification using each of these new distance measures and the three normalization options from Question 3. Discuss how the new distance metrics compare to Euclidean distance and how each metric is affected by normalization.

### 4.3
Implement either of the two K-NN weighting strategies discussed in lecture (inverse linear distance or inverse distance). Compare the performance of the weighted and majority vote models for a few different values of K. In your write-up, discuss how weighting strategy and the value of K affect the model's decisions.

### 4.4
Measure the empirical distribution of class labels in the training dataset (what percentage of the training data comes from each class). Then evaluate the distribution of labels predicted by your K-NN model for the test data, for a range of values for K. Does the class distribution of the predicted labels match the class distribution of the training data? Explain why or why not.